In [1]:
import pandas as pd
import warnings

# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar

# ==================== CELL 1: ĐỌC DỮ LIỆU ====================
import pandas as pd
import os

print("🔄 Bắt đầu đọc dữ liệu Parquet...")
parquet_folder = r"C:\Khue\TDN\data\interim"  # <-- Đảm bảo đường dẫn này chính xác
file_name = "BCSVH_Bac_2024.parquet"
file_path = os.path.join(parquet_folder, file_name)

try:
    # Đọc file cụ thể
    if os.path.exists(file_path):
        df_all = pd.read_parquet(file_path, engine="pyarrow")
        print(f"✅ Đã đọc xong file {file_name}")
        print(f"👉 Tổng số dòng dữ liệu ban đầu: {df_all.shape[0]:,}")
    else:
        print(f"⚠️ Không tìm thấy file: {file_path}")
        df_all = pd.DataFrame()  # Tạo DataFrame rỗng để code không lỗi ở các bước sau
except Exception as e:
    print(f"❌ Lỗi khi đọc file {file_path}: {e}")
    df_all = pd.DataFrame()  # Tạo DataFrame rỗng

# ==================== CELL 2: CHỌN CỘT VÀ ĐỔI TÊN ====================
if not df_all.empty:
    # Chọn các cột cần thiết và TẠO BẢN SAO (.copy()) để tránh SettingWithCopyWarning
    required_cols = ["CTDL", "NMTD", "ENDTIME", "CS", "P_rated"]
    if all(col in df_all.columns for col in required_cols):
        df_sanluong = df_all[required_cols].copy()
        # Đổi tên cột ENDTIME thành TIME
        df_sanluong.rename(columns={"ENDTIME": "TIME"}, inplace=True)
    else:
        print(f"❌ Thiếu cột cần thiết trong dữ liệu. Cần có: {required_cols}")
        missing = [col for col in required_cols if col not in df_all.columns]
        print(f"   -> Các cột bị thiếu: {missing}")
        df_sanluong = pd.DataFrame()  # Tạo rỗng để tránh lỗi sau
else:
    print("ℹ️ Bỏ qua bước chuẩn bị dữ liệu sản lượng do không có dữ liệu đầu vào.")
    df_sanluong = pd.DataFrame()

# ==================== CELL 3: XỬ LÝ THỜI GIAN, TẠO UI VÀ VẼ BIỂU ĐỒ ====================

# Chỉ tiếp tục nếu df_sanluong có dữ liệu
if not df_sanluong.empty:
    print("🔄 Xử lý thời gian và tối ưu DataFrame...")
    # ========== TỐI ƯU HÓA: XỬ LÝ DATETIME VÀ ĐẶT INDEX ==========
    # 1. Đảm bảo cột TIME là datetime
    try:
        df_sanluong["TIME"] = pd.to_datetime(df_sanluong["TIME"])
    except Exception as e:
        print(f"❌ Lỗi chuyển đổi cột TIME sang datetime: {e}")
        # Cân nhắc dừng hoặc xử lý lỗi khác tại đây
        df_sanluong = pd.DataFrame()  # Đặt lại thành rỗng nếu không xử lý được time

    if not df_sanluong.empty:
        # 2. Tạo các cột thời gian để index
        df_sanluong["YEAR"] = df_sanluong["TIME"].dt.year
        df_sanluong["MONTH_NUM"] = df_sanluong["TIME"].dt.month
        df_sanluong["DAY_NUM"] = df_sanluong["TIME"].dt.day

        # 3. Đặt MultiIndex (KHÔNG bao gồm 'TIME' vì cần nó làm cột cho trục X)
        #    Index sẽ giúp tăng tốc lọc bằng .loc
        index_cols = ["CTDL", "NMTD", "YEAR", "MONTH_NUM", "DAY_NUM"]
        # Kiểm tra sự tồn tại của các cột trước khi đặt index
        if all(col in df_sanluong.columns for col in index_cols):
            # Giữ lại các cột cần thiết cho plot ('TIME', 'CS', 'MADIEMDO') làm cột thường
            df_sanluong.set_index(index_cols, inplace=True)

            # 4. Sắp xếp Index - BƯỚC QUAN TRỌNG để .loc nhanh
            df_sanluong.sort_index(inplace=True)
            print("✅ Đã xử lý thời gian và tối ưu index DataFrame.")
        else:
            print("❌ Thiếu cột cần thiết để tạo index tối ưu.")
            missing_idx_cols = [
                col for col in index_cols if col not in df_sanluong.columns
            ]
            print(f"   -> Các cột index bị thiếu: {missing_idx_cols}")
            # Nếu muốn vẫn chạy nhưng không tối ưu, có thể bỏ qua set_index và sort_index
            # Nếu không, đặt df_sanluong thành rỗng
            df_sanluong = pd.DataFrame()


# --- Các phần còn lại chỉ chạy nếu df_sanluong sau khi xử lý vẫn có dữ liệu ---
if not df_sanluong.empty:
    print("🔄 Tạo giao diện tương tác...")
    # ========== DANH SÁCH CHO DROPDOWN ==========
    # Lấy từ index thay vì cột sau khi đã set_index
    try:
        ctdl_list = sorted(df_sanluong.index.get_level_values("CTDL").unique())
        year_list = sorted(df_sanluong.index.get_level_values("YEAR").unique())
    except Exception as e:
        print(f"❌ Lỗi khi lấy danh sách CTDL/Year từ index: {e}")
        ctdl_list = []
        year_list = []

    # ========== HÀM TẠO DROPDOWN ==========
    def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
        # Đảm bảo options không phải là None hoặc kiểu không phù hợp
        safe_options = list(options) if options is not None else []
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            layout=widgets.Layout(width=width, margin=margin),
            style={"description_width": "auto"},
            disabled=not bool(safe_options),  # Vô hiệu hóa nếu không có lựa chọn
        )

    # ========== DROPDOWNS ==========
    ctdl_dropdown_month = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_month = make_dropdown([], "Nhà máy:", "300px")  # Bắt đầu rỗng
    year_dropdown_month = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_month = make_dropdown([], "Tháng:", "120px")
    month_dropdown_month.disabled = True  # Bắt đầu ở trạng thái disabled
    day_placeholder = widgets.Label(
        value="", layout=widgets.Layout(width="120px", margin="0px 20px 0px 0px")
    )  # Để căn chỉnh

    ctdl_dropdown_day = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_day = make_dropdown([], "Nhà máy:", "300px")  # Bắt đầu rỗng
    year_dropdown_day = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_day = make_dropdown([], "Tháng:", "120px")
    month_dropdown_day.disabled = True  # Bắt đầu ở trạng thái disabled
    day_dropdown_day = make_dropdown(
        [], "Ngày:", "120px"
    )  # Bắt đầu rỗng, sẽ được cập nhật

    # ========== NHÓM WIDGET ==========
    month_widgets = [
        ctdl_dropdown_month,
        nmtd_dropdown_month,
        year_dropdown_month,
        month_dropdown_month,
    ]
    day_widgets = [
        ctdl_dropdown_day,
        nmtd_dropdown_day,
        year_dropdown_day,
        month_dropdown_day,
        day_dropdown_day,
    ]

    # ========== CẬP NHẬT NHÀ MÁY (Tối ưu bằng .loc trên index) ==========
    def update_nmtd(ctdl_value, dropdown, update_month_callback=None):
        options = []  # Mặc định là rỗng
        if ctdl_value and not df_sanluong.empty:
            try:
                # Lấy danh sách NMTD duy nhất tại level 'NMTD' của index, ứng với ctdl_value đã chọn
                options = sorted(
                    df_sanluong.loc[ctdl_value].index.get_level_values("NMTD").unique()
                )
            except KeyError:
                # Trường hợp ctdl_value không tồn tại trong index
                options = []
            except Exception as e:
                # Bắt các lỗi khác có thể xảy ra
                print(f" Lỗi khi cập nhật NMTD cho CTDL '{ctdl_value}': {e}")
                options = []

        # Cập nhật options và trạng thái disabled
        dropdown.options = options
        dropdown.disabled = not bool(options)
        # Nếu options cập nhật thành rỗng, reset giá trị về None
        if not options:
            dropdown.value = None
        else:
            # Đặt giá trị mặc định nếu chưa có giá trị hoặc giá trị hiện tại không hợp lệ
            if dropdown.value not in options:
                dropdown.value = options[0]

        # Gọi callback để cập nhật tháng nếu được cung cấp
        if update_month_callback and dropdown.value is not None:
            update_month_callback()

    # Gắn sự kiện observe và gọi lần đầu để khởi tạo NMTD
    # Sử dụng lambda để truyền đúng dropdown cần cập nhật và callback cập nhật tháng
    ctdl_dropdown_month.observe(
        lambda change: update_nmtd(
            change["new"], nmtd_dropdown_month, update_month_options_month
        ),
        names="value",
    )
    ctdl_dropdown_day.observe(
        lambda change: update_nmtd(
            change["new"], nmtd_dropdown_day, update_month_options_day
        ),
        names="value",
    )

    # Khởi tạo NMTD ban đầu (nếu ctdl_dropdown có giá trị)
    # Khởi tạo tháng ban đầu (nếu đã có NMTD)
    # ========== CẬP NHẬT THÁNG (dựa trên dữ liệu thực tế) ==========
    def update_month_options(year_value, ctdl_value, nmtd_value, dropdown):
        options = []  # Mặc định là rỗng
        if all([ctdl_value, nmtd_value, year_value]) and not df_sanluong.empty:
            try:
                # Lấy danh sách tháng có dữ liệu cho CTDL, NMTD và năm đã chọn
                idx_partial = (ctdl_value, nmtd_value, year_value)
                # Kiểm tra xem idx_partial có tồn tại trong index không
                if any(idx[0:3] == idx_partial for idx in df_sanluong.index.values):
                    # Lấy các tháng duy nhất cho tổ hợp CTDL, NMTD, năm đã chọn
                    available_months = sorted(
                        df_sanluong.xs(idx_partial, level=["CTDL", "NMTD", "YEAR"])
                        .index.get_level_values("MONTH_NUM")
                        .unique()
                    )
                    options = available_months
            except Exception as e:
                print(f" Lỗi khi cập nhật danh sách tháng: {e}")
                options = []

        # Cập nhật options và trạng thái disabled
        dropdown.options = options
        dropdown.disabled = not bool(options)
        # Nếu options cập nhật thành rỗng, reset giá trị về None
        if not options:
            dropdown.value = None
        elif dropdown.value not in options:
            dropdown.value = options[0]  # Chọn tháng đầu tiên có sẵn

    # Tạo hàm callback cho việc cập nhật tháng
    def update_month_options_month(*args):
        update_month_options(
            year_dropdown_month.value,
            ctdl_dropdown_month.value,
            nmtd_dropdown_month.value,
            month_dropdown_month,
        )

    def update_month_options_day(*args):
        update_month_options(
            year_dropdown_day.value,
            ctdl_dropdown_day.value,
            nmtd_dropdown_day.value,
            month_dropdown_day,
        )

    # Đăng ký callback cho việc cập nhật tháng
    year_dropdown_month.observe(update_month_options_month, names="value")
    nmtd_dropdown_month.observe(update_month_options_month, names="value")

    year_dropdown_day.observe(update_month_options_day, names="value")
    nmtd_dropdown_day.observe(update_month_options_day, names="value")

    # Khởi tạo giá trị ban đầu cho Nhà máy dựa trên CTDL đã chọn
    if ctdl_dropdown_month.value:
        update_nmtd(
            ctdl_dropdown_month.value, nmtd_dropdown_month, update_month_options_month
        )
    if ctdl_dropdown_day.value:
        update_nmtd(
            ctdl_dropdown_day.value, nmtd_dropdown_day, update_month_options_day
        )

    # ========== GIỚI HẠN NGÀY ==========
    # ========== GIỚI HẠN NGÀY (dựa trên dữ liệu thực tế) ==========
    def update_day_options(*args):
        ctdl = ctdl_dropdown_day.value
        nmtd = nmtd_dropdown_day.value
        year = year_dropdown_day.value
        month = month_dropdown_day.value
        current_day_value = day_dropdown_day.value  # Lưu giá trị ngày hiện tại

        options = []  # Mặc định là rỗng
        if all([ctdl, nmtd, year, month]) and not df_sanluong.empty:
            try:
                # Lấy danh sách ngày có dữ liệu cho CTDL, NMTD, năm và tháng đã chọn
                idx_partial = (ctdl, nmtd, year, month)
                # Kiểm tra xem idx_partial có tồn tại trong index không
                if any(idx[0:4] == idx_partial for idx in df_sanluong.index.values):
                    # Lấy các ngày duy nhất cho tổ hợp CTDL, NMTD, năm, tháng đã chọn
                    available_days = sorted(
                        df_sanluong.xs(
                            idx_partial, level=["CTDL", "NMTD", "YEAR", "MONTH_NUM"]
                        )
                        .index.get_level_values("DAY_NUM")
                        .unique()
                    )
                    options = available_days
            except Exception as e:
                print(f" Lỗi khi cập nhật danh sách ngày: {e}")
                options = []

        # Cập nhật options và trạng thái disabled
        day_dropdown_day.options = options
        day_dropdown_day.disabled = not bool(options)
        # Nếu options cập nhật thành rỗng, reset giá trị về None
        if not options:
            day_dropdown_day.value = None
        elif current_day_value in options:
            day_dropdown_day.value = (
                current_day_value  # Giữ nguyên ngày đã chọn nếu còn hợp lệ
            )
        else:
            day_dropdown_day.value = (
                options[0] if options else None
            )  # Chọn ngày đầu tiên có sẵn

    # Gắn sự kiện observe và gọi lần đầu để khởi tạo Ngày
    year_dropdown_day.observe(update_day_options, names="value")
    month_dropdown_day.observe(update_day_options, names="value")
    update_day_options()  # Gọi để khởi tạo lần đầu

    # ========== VÙNG OUTPUT CHO BIỂU ĐỒ ==========
    # Đặt trước để hàm plot_filtered có thể truy cập
    out1 = widgets.Output()

    # ========== VẼ BIỂU ĐỒ (Tối ưu bằng .loc trên index) ==========
    def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
        # Xóa output cũ trước khi bắt đầu
        with out1:
            clear_output(wait=True)  # wait=True giảm nhấp nháy

            # Kiểm tra đầu vào cơ bản
            if not all([ctdl, nmtd, year, month]):
                print("Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
                return
            if mode == "day" and not day:
                print("Vui lòng chọn Ngày để xem theo ngày.")
                return

            filtered_df = pd.DataFrame()  # Khởi tạo DataFrame rỗng

            try:
                if mode == "month":
                    # Lọc bằng .loc trên index (CTDL, NMTD, YEAR, MONTH_NUM)
                    # reset_index() để lấy lại các cột index và các cột dữ liệu thường
                    idx_query = (ctdl, nmtd, year, month)
                    if (
                        idx_query in df_sanluong.index
                    ):  # Kiểm tra nhanh sự tồn tại của key chính
                        filtered_df = df_sanluong.loc[idx_query].reset_index()
                    title_ext = f"{month}/{year}"
                else:  # mode == 'day'
                    # Lọc bằng .loc trên index (CTDL, NMTD, YEAR, MONTH_NUM, DAY_NUM)
                    idx_query = (ctdl, nmtd, year, month, day)
                    if idx_query in df_sanluong.index:
                        filtered_df = df_sanluong.loc[idx_query].reset_index()
                    title_ext = f"{day}/{month}/{year}"

            except KeyError:
                # Xử lý trường hợp tổ hợp index không tồn tại (dù đã kiểm tra sơ bộ)
                print(f"ℹ️ Không tìm thấy dữ liệu chính xác cho lựa chọn (KeyError).")
                filtered_df = pd.DataFrame()  # Đảm bảo là DataFrame rỗng
            except Exception as e:
                print(f"❌ Lỗi khi lọc dữ liệu bằng .loc: {e}")
                filtered_df = pd.DataFrame()  # Đảm bảo là DataFrame rỗng

            # Kiểm tra lại lần nữa sau khi lọc và reset_index
            if filtered_df.empty:
                print("ℹ️ Không có dữ liệu cho lựa chọn này.")
                return  # Thoát hàm nếu không có dữ liệu

            # --- Vẽ biểu đồ (Giữ nguyên cấu hình) ---
            try:
                # Sắp xếp lại theo TIME sau khi reset_index (quan trọng cho biểu đồ đường)
                filtered_df = filtered_df.sort_values("TIME")

                # Đã loại bỏ tham số color="MADIEMDO"
                # Get P_rated value for the selected plant
                p_rated = None
                try:
                    # Get P_rated from the original df_all DataFrame for the selected NMTD
                    p_rated = df_all[(df_all['CTDL'] == ctdl) & (df_all['NMTD'] == nmtd)]['P_rated'].iloc[0]
                except Exception as e:
                    print(f"Không thể lấy được giá trị P_rated: {e}")
                
                # Create the line chart
                fig1 = px.line(
                    filtered_df,
                    x="TIME",
                    y="CS",
                    render_mode="webgl",
                    markers=True if mode == "day" else False,
                )

                fig1.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>BCSVH Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất (MW)",
                    xaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    yaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        rangemode="tozero",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )

                # ✅ Tên trục X bằng annotation
                fig1.add_annotation(
                    text="<b>Thời gian</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )

                # Thay đổi màu đường và độ dày
                fig1.update_traces(
                    line=dict(width=2, color="#3366ff"),  # Đặt màu mặc định
                    hovertemplate="Công suất: %{y}",  # Cập nhật mẫu hiển thị khi hover
                )
                
                # Thêm đường thẳng nét đứt màu đỏ cho P_rated
                if p_rated is not None:
                    fig1.add_shape(
                        type="line",
                        x0=0,
                        y0=p_rated,
                        x1=1,
                        y1=p_rated,
                        xref="paper",
                        yref="y",
                        line=dict(
                            color="red",
                            width=2,
                            dash="dash",
                        )
                    )
                    
                    # Thêm giá trị P_rated như một "tick label" bằng annotation
                    fig1.add_annotation(
                        xref="paper", # Tham chiếu đến paper cho tọa độ x
                        yref="y",     # Tham chiếu đến giá trị trục y cho tọa độ y
                        x=1,       # Đặt vị trí góc phải biểu đồ
                        y=p_rated*1.05, # Đặt vị trí trên đường thẳng P_rated
                        text=f"<b>Công suất đặt: {p_rated} MW</b>", # Văn bản nhãn với thêm chú thích
                        showarrow=False,
                        xanchor="right", # Căn lề phải cho văn bản
                        yanchor="middle",# Căn lề giữa theo chiều dọc tại giá trị y
                        font=dict(color="black", size=12), # Định dạng giống màu đường line
                        bgcolor="rgba(255, 255, 255, 0.7)", # Nền trắng mờ để dễ đọc
                        bordercolor="black",
                        borderwidth=1,
                        borderpad=4
                    )

                fig1.show()  # Hiển thị biểu đồ trong Output 'out'

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ Plotly: {e}")
            # Có thể hiển thị lỗi chi tiết hơn nếu cần debug

    # ========== HÀM XỬ LÝ SỰ KIỆN THAY ĐỔI DROPDOWN ==========
    def on_change_month(change):
        # Kiểm tra xem các widget cần thiết có giá trị không
        if all(
            w.value is not None
            for w in [
                ctdl_dropdown_month,
                nmtd_dropdown_month,
                year_dropdown_month,
                month_dropdown_month,
            ]
        ):
            # Chỉ gọi plot nếu là sự thay đổi có ý nghĩa (không phải reset về None)
            if change is None or change["new"] != change["old"]:
                plot_filtered(
                    "month",
                    ctdl_dropdown_month.value,
                    nmtd_dropdown_month.value,
                    year_dropdown_month.value,
                    month_dropdown_month.value,
                )
        # else: # Debugging (optional)
        # print("Debug Month: Một hoặc nhiều widget tháng chưa có giá trị.")

    def on_change_day(change):
        if all(
            w.value is not None
            for w in [
                ctdl_dropdown_day,
                nmtd_dropdown_day,
                year_dropdown_day,
                month_dropdown_day,
                day_dropdown_day,
            ]
        ):
            if change is None or change["new"] != change["old"]:
                plot_filtered(
                    "day",
                    ctdl_dropdown_day.value,
                    nmtd_dropdown_day.value,
                    year_dropdown_day.value,
                    month_dropdown_day.value,
                    day_dropdown_day.value,
                )
        # else: # Debugging (optional)
        # print("Debug Day: Một hoặc nhiều widget ngày chưa có giá trị.")

    # ========== HÀM TẮT / BẬT OBSERVER ==========
    # Giữ nguyên các hàm này, chúng quan trọng cho logic chuyển tab
    _observed_callbacks_month = {}
    _observed_callbacks_day = {}

    def disable_observe(widgets, callback_map, callback_func):
        for w in widgets:
            try:
                # Lưu trữ hàm callback để có thể gắn lại đúng hàm
                callback_map[w] = callback_func
                w.unobserve(callback_func, names="value")
                # print(f"Unobserved {w.description}") # Debug
            except ValueError:
                # print(f"Callback already removed for {w.description}") # Debug
                pass  # Bỏ qua nếu callback đã được gỡ bỏ trước đó

    def enable_observe(widgets, callback_map):
        for w in widgets:
            if w in callback_map:
                try:
                    # Chỉ observe nếu widget không bị disabled
                    if not w.disabled:
                        w.observe(callback_map[w], names="value")
                        # print(f"Observed {w.description}") # Debug
                    # else:
                    # print(f"Skipped observe on disabled {w.description}") # Debug

                except Exception as e:
                    print(f"Error observing {w.description}: {e}")  # Debug

    # ========== GẮN OBSERVER BAN ĐẦU ==========
    # Thay vì lặp, gọi enable_observe để gắn kèm kiểm tra disabled
    # Tạo map trước khi gọi enable_observe
    for w in month_widgets:
        _observed_callbacks_month[w] = on_change_month
    for w in day_widgets:
        _observed_callbacks_day[w] = on_change_day

    enable_observe(month_widgets, _observed_callbacks_month)
    enable_observe(day_widgets, _observed_callbacks_day)

    # ========== TẠO GIAO DIỆN ==========
    tab_thang = widgets.HBox(
        [
            ctdl_dropdown_month,
            nmtd_dropdown_month,
            year_dropdown_month,
            month_dropdown_month,
            day_placeholder,
        ],
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )  # Căn chỉnh đáy widget

    tab_ngay = widgets.HBox(
        [
            ctdl_dropdown_day,
            nmtd_dropdown_day,
            year_dropdown_day,
            month_dropdown_day,
            day_dropdown_day,
        ],
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )  # Căn chỉnh đáy widget

    tabs1 = widgets.Tab(children=[tab_thang, tab_ngay])
    tabs1.set_title(0, "📅 Xem theo tháng")
    tabs1.set_title(1, "🗓️ Xem theo ngày")

    # ========== XỬ LÝ CHUYỂN TAB ==========
    def on_tab_change(change):
        if change["name"] == "selected_index":
            new_tab_index = change["new"]
            print(f"\n🔄 Chuyển sang tab: {tabs1.get_title(new_tab_index)}")

            if new_tab_index == 1:  # Chuyển sang tab NGÀY
                print("   Đồng bộ hóa lựa chọn từ Tháng sang Ngày...")
                # 1. Tắt observer của tab Ngày để tránh kích hoạt khi cập nhật giá trị
                disable_observe(day_widgets, _observed_callbacks_day, on_change_day)

                # 2. Đồng bộ giá trị từ tab Tháng sang tab Ngày
                ctdl_dropdown_day.value = ctdl_dropdown_month.value
                # update_nmtd sẽ được trigger bởi observe của ctdl_day khi bật lại,
                # nhưng ta cần gọi update trực tiếp để lấy đúng NMTD list trước
                update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)
                # Chỉ gán NMTD nếu giá trị tháng tồn tại trong options mới
                if nmtd_dropdown_month.value in nmtd_dropdown_day.options:
                    nmtd_dropdown_day.value = nmtd_dropdown_month.value
                else:
                    nmtd_dropdown_day.value = (
                        nmtd_dropdown_day.options[0]
                        if nmtd_dropdown_day.options
                        else None
                    )  # Chọn cái đầu tiên hoặc None

                year_dropdown_day.value = year_dropdown_month.value
                month_dropdown_day.value = month_dropdown_month.value
                # update_day_options sẽ tự động được gọi khi month/year thay đổi (nếu observe đang bật)
                # Gọi lại để chắc chắn ngày được cập nhật đúng
                update_day_options()
                # Đặt ngày mặc định là 1 nếu có trong options
                if 1 in day_dropdown_day.options:
                    day_dropdown_day.value = 1
                elif day_dropdown_day.options:
                    day_dropdown_day.value = day_dropdown_day.options[0]
                else:
                    day_dropdown_day.value = None
                # 3. Bật lại observer cho tab Ngày
                enable_observe(day_widgets, _observed_callbacks_day)

                # 4. Kích hoạt vẽ lại biểu đồ cho tab Ngày với giá trị mới
                print("   Vẽ biểu đồ cho tab Ngày...")
                on_change_day(None)  # None để chỉ ra đây là gọi thủ công

            elif new_tab_index == 0:  # Chuyển sang tab THÁNG
                print("   Đồng bộ hóa lựa chọn từ Ngày sang Tháng...")
                # 1. Tắt observer của tab Tháng
                disable_observe(
                    month_widgets, _observed_callbacks_month, on_change_month
                )

                # 2. Đồng bộ giá trị từ tab Ngày sang tab Tháng
                ctdl_dropdown_month.value = ctdl_dropdown_day.value
                update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
                if nmtd_dropdown_day.value in nmtd_dropdown_month.options:
                    nmtd_dropdown_month.value = nmtd_dropdown_day.value
                else:
                    nmtd_dropdown_month.value = (
                        nmtd_dropdown_month.options[0]
                        if nmtd_dropdown_month.options
                        else None
                    )

                year_dropdown_month.value = year_dropdown_day.value
                month_dropdown_month.value = month_dropdown_day.value

                # 3. Bật lại observer cho tab Tháng
                enable_observe(month_widgets, _observed_callbacks_month)

                # 4. Kích hoạt vẽ lại biểu đồ cho tab Tháng
                print("   Vẽ biểu đồ cho tab Tháng...")
                on_change_month(None)

    tabs1.observe(on_tab_change, names="selected_index")

    # ========== HIỂN THỊ ==========
    print("✅ Giao diện sẵn sàng!")
    display(tabs1, out1)

    # ========== VẼ BIỂU ĐỒ BAN ĐẦU ==========
    # Gọi on_change_month để vẽ biểu đồ lần đầu tiên khi cell chạy xong (cho tab tháng mặc định)

    on_change_month(None)

elif df_all.empty:
    print("\n❌ Không có dữ liệu đầu vào từ file Parquet để xử lý và vẽ biểu đồ.")
else:
    print(
        "\n❌ Đã xảy ra lỗi trong quá trình chuẩn bị dữ liệu. Không thể tạo giao diện."
    )

🔄 Bắt đầu đọc dữ liệu Parquet...
✅ Đã đọc xong file BCSVH_Bac_2024.parquet
👉 Tổng số dòng dữ liệu ban đầu: 3,918,768
🔄 Xử lý thời gian và tối ưu DataFrame...
✅ Đã xử lý thời gian và tối ưu index DataFrame.
🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()